In [3]:
%pip install tensorflow


  Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp310-cp310-win_amd64.whl (272.8 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.8.0-cp310-cp310-win_amd64.whl (2.6 MB)
  Using cached jax-0.4.12-py3-none-any.whl
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.54.2-cp310-cp310-win_amd64.whl (4.1 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-win_amd64.whl (938 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
df_98 = pd.read_csv('NBA_data/nba_games_1998.csv', index_col=0)
df_99_22 = pd.read_csv('NBA_data//nba_games_1999_2022.csv', index_col=0)
df_23 = pd.read_csv('NBA_data//nba_games_2023.csv', index_col=0)
df = pd.concat([df_98, df_99_22, df_23])

In [11]:
df = df.sort_values('date')
df = df.reset_index(drop=True)
del df['index_opp']

In [12]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

In [14]:
df = df.groupby('team', group_keys=False).apply(add_target)

In [16]:
df['target'][pd.isnull(df['target'])] = 2
df['target'] = df['target'].astype(int, errors='ignore')

C:\Users\USER\AppData\Local\Temp\ipykernel_18880\1965879857.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'][pd.isnull(df['target'])] = 2


In [17]:
df['won'].value_counts()

False    27741
True     27741
Name: won, dtype: int64

In [18]:
df['target'].value_counts()

1    27723
0    27722
2       37
Name: target, dtype: int64

In [20]:
nulls = pd.isnull(df)
nulls = nulls.sum()
nulls = nulls[nulls > 0]
nulls

Series([], dtype: int64)

In [26]:
df.dropna()

,fta,efg%,usg%,trb%,drtg,ast%,3pa,orb%,ft%,stl%,...,ft_opp,3p_opp,tov_opp,team_opp,total_opp,home_opp,season,date,won,target
0,29,0.424,100.0,44.1,98.7,48.5,8,34.8,0.621,12.9,...,11,5,23,BOS,92,1,1998,1997-10-31,False,1
1,20,0.416,100.0,54.3,103.4,69.4,13,47.1,0.700,8.0,...,25,1,16,DAL,90,0,1998,1997-10-31,False,1
2,26,0.473,100.0,53.0,99.2,63.6,12,40.9,0.500,13.1,...,15,4,18,SEA,91,0,1998,1997-10-31,False,1
3,26,0.481,100.0,47.0,90.5,58.3,13,33.3,0.577,17.4,...,13,4,25,POR,83,1,1998,1997-10-31,True,0
4,37,0.533,100.0,48.8,105.4,73.7,8,29.3,0.811,11.6,...,10,10,19,LAC,100,0,1998,1997-10-31,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55477,22,0.593,100.0,55.1,127.8,59.0,28,28.1,0.864,8.1,...,18,17,11,MIA,111,0,2023,2023-06-04,False,1
55478,27,0.544,100.0,63.7,103.8,68.3,18,36.1,0.815,3.3,...,15,11,4,MIA,94,1,2023,2023-06-07,True,1
55479,19,0.429,100.0,36.3,120.4,58.8,35,18.2,0.789,7.7,...,22,5,13,DEN,109,0,2023,2023-06-07,False,0
55480,21,0.582,100.0,47.9,107.2,66.7,28,14.7,0.762,12.4,...,17,8,14,MIA,95,1,2023,2023-06-09,True,2


In [31]:
removed_columns = ['season', 'date', 'won', 'target', 'team', 'team_opp']

In [32]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [54]:
teams = list(df.team.unique())

In [57]:
df_teams = []
for team in teams:
    df_team = df[df['team'] == team]
    df_team = df_team.reset_index(drop=True)
    df_teams.append(df_team)
df_teams[0]

,fta,efg%,usg%,trb%,drtg,ast%,3pa,orb%,ft%,stl%,...,ft_opp,3p_opp,tov_opp,team_opp,total_opp,home_opp,season,date,won,target
0,29,0.424,100.0,44.1,98.7,48.5,8,34.8,0.621,12.9,...,11,5,23,BOS,92,1,1998,1997-10-31,False,1
1,22,0.500,100.0,51.1,78.9,72.5,4,36.4,0.591,16.0,...,15,1,25,PHI,74,0,1998,1997-11-01,True,1
2,24,0.333,100.0,51.7,81.5,63.6,17,37.1,0.625,11.8,...,15,0,23,SAS,83,0,1998,1997-11-03,True,1
3,25,0.487,100.0,54.2,95.8,71.1,7,38.1,0.680,3.5,...,30,1,11,ORL,81,0,1998,1997-11-05,True,0
4,16,0.413,100.0,43.4,91.5,59.4,8,15.4,0.750,9.1,...,19,3,14,ATL,80,1,1998,1997-11-07,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,11,0.500,100.0,41.8,111.0,77.8,33,11.1,0.818,10.6,...,9,18,12,MIL,105,1,2023,2023-04-05,False,1
1844,18,0.585,100.0,53.8,112.9,52.2,31,18.2,0.667,6.0,...,10,18,9,DAL,112,1,2023,2023-04-07,True,1
1845,22,0.453,100.0,44.9,84.1,52.5,30,20.0,0.773,15.6,...,10,9,24,DET,81,0,2023,2023-04-09,True,1
1846,22,0.529,100.0,41.9,109.3,47.6,26,19.0,0.818,9.4,...,18,11,14,TOR,105,1,2023,2023-04-12,True,0


In [96]:
X = df_teams[1].drop(removed_columns, axis=1).values
y = df_teams[1]['target'].values

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [98]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [99]:
model = Sequential()

In [100]:
model.add(Dense(64, activation='relu', input_shape=(72,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [101]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [102]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Pérdida: ", loss)
print("Precisión: ", accuracy)

1/1 [==============================] - 1s 1s/step - loss: 0.7665 - accuracy: 0.5556
Pérdida:  0.766521155834198
Precisión:  0.5555555820465088


In [15]:
df

,fta,efg%,usg%,trb%,drtg,ast%,3pa,orb%,ft%,stl%,...,ft_opp,3p_opp,tov_opp,team_opp,total_opp,home_opp,season,date,won,target
0,29,0.424,100.0,44.1,98.7,48.5,8,34.8,0.621,12.9,...,11,5,23,BOS,92,1,1998,1997-10-31,False,True
1,20,0.416,100.0,54.3,103.4,69.4,13,47.1,0.700,8.0,...,25,1,16,DAL,90,0,1998,1997-10-31,False,True
2,26,0.473,100.0,53.0,99.2,63.6,12,40.9,0.500,13.1,...,15,4,18,SEA,91,0,1998,1997-10-31,False,True
3,26,0.481,100.0,47.0,90.5,58.3,13,33.3,0.577,17.4,...,13,4,25,POR,83,1,1998,1997-10-31,True,False
4,37,0.533,100.0,48.8,105.4,73.7,8,29.3,0.811,11.6,...,10,10,19,LAC,100,0,1998,1997-10-31,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55477,22,0.593,100.0,55.1,127.8,59.0,28,28.1,0.864,8.1,...,18,17,11,MIA,111,0,2023,2023-06-04,False,True
55478,27,0.544,100.0,63.7,103.8,68.3,18,36.1,0.815,3.3,...,15,11,4,MIA,94,1,2023,2023-06-07,True,True
55479,19,0.429,100.0,36.3,120.4,58.8,35,18.2,0.789,7.7,...,22,5,13,DEN,109,0,2023,2023-06-07,False,False
55480,21,0.582,100.0,47.9,107.2,66.7,28,14.7,0.762,12.4,...,17,8,14,MIA,95,1,2023,2023-06-09,True,NaN
